In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from find_walking_frames import find_walking_data
from pre_process import find_period_acf


In [ ]:
directory = 'data/'

subject_num = '4'

sub_walks = find_walking_data(subject_num, directory)

keys = sub_walks.keys()

sub_choice = random.choice(list(keys))

length = len(sub_walks[sub_choice])



In [ ]:
df = sub_walks[sub_choice][random.randint(0, length-1)]

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
from datetime import timedelta

def conv_acceleration(num):
    return 2*num/32768


df_accel = pd.DataFrame(None)

for column in df.columns:
    if 'acc' in column:
        df_accel[column] = df[column].apply(conv_acceleration)

# delta = timedelta(seconds = 1/56.35)


# df_accel['time'] = (df.index - df.index[0]) * delta

# df_accel.set_index('time', inplace = True)

df_accel.head()

The acceleration data can be converted to gravitational accelleration by taking $2x/32768$.

In [ ]:
df_accel_rs = df_accel[['acc_rs_x', 'acc_rs_y', 'acc_rs_z']]

In [ ]:
fig, axes = plt.subplots(3,1, sharex = True)
for ax,column in zip(axes, df_accel_rs.columns):
    ax.plot(df_accel_rs.index, df_accel_rs[column])
    ax.set_ylim(-2,2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(df_accel_rs.index, df_accel_rs['acc_rs_z'])
ax.set_ylim(-2,2)

In [ ]:
df = df_accel_rs['acc_rs_z']

period = find_period_acf(df, 150)

num_strides = len(df)//period + 1

In [ ]:
from gtda.time_series import takens_embedding_optimal_parameters
from random import randint

num_periods = 4

wind_start = randint(0,num_strides - num_periods)

series = np.asarray(df_accel_rs['acc_rs_z'][wind_start*period: wind_start*period + num_periods*period])
max_embed_dim = 10
max_delay = 5
stride = 1

optimal_time_delay, optimal_embedding_dimension = takens_embedding_optimal_parameters(
    series, max_delay, max_embed_dim, stride=stride
    )

print(f"Optimal embedding time delay based on mutual information: {optimal_time_delay}")
print(f"Optimal embedding dimension based on false nearest neighbors: {optimal_embedding_dimension}")


In [ ]:
from gtda.time_series import SingleTakensEmbedding

TE = SingleTakensEmbedding(time_delay=optimal_time_delay, dimension=optimal_embedding_dimension, stride=stride)
series_embedded = TE.fit_transform((series.reshape(-1,1)))

In [ ]:
from sklearn.decomposition import PCA
from gtda.plotting import plot_point_cloud

if optimal_embedding_dimension >=3:
    pca = PCA(n_components=3)

    series_embedded_pca = pca.fit_transform(series_embedded)
else:
    series_embedded_pca = series_embedded

plot_point_cloud(series_embedded_pca)

In [ ]:
series_embedded_data = np.array(series_embedded_pca)


In [ ]:
from gtda.homology import WeakAlphaPersistence

homology_dims = [0, 1]

WA = WeakAlphaPersistence(homology_dimensions = homology_dims)
X_diagrams = WA.fit_transform(series_embedded_data[None,:,:],) 

WA.plot(X_diagrams)

In [ ]:
from gtda.diagrams import Scaler
scaling = Scaler()
X_diag_scaled = scaling.fit_transform(X_diagrams)
WA.plot(X_diag_scaled)

In [ ]:
from gtda.diagrams import PersistenceEntropy

entropy = PersistenceEntropy(normalize = True, nan_fill_value=-10)

entropy.fit_transform(X_diag_scaled)